In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
print("Working directory:", os.getcwd())
path = os.path.abspath('../data/cleaned_data/infrastructure/BMMS_overview.xlsx')
print("Absolute path being used:", path)
print("Exists?", os.path.exists(path))

# # Force the working dir to the 'model' folder
# os.chdir(r"C:\Users\domho\OneDrive\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\model")

Working directory: C:\Users\tobia\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\model
Absolute path being used: C:\Users\tobia\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\data\cleaned_data\infrastructure\BMMS_overview.xlsx
Exists? True


In [3]:
bridges = pd.read_excel('../data/cleaned_data/infrastructure/BMMS_overview.xlsx')
roads = pd.read_csv('../data/cleaned_data/infrastructure/_roads3.csv')

In [4]:
# Filter only bridges related to road N1
bridges_n1 = bridges[bridges['road'] == 'N1']

# Rename columns to align with roads DataFrame (type -> model_type)
bridges_n1 = bridges_n1.rename(columns={"type": "model_type"})

bridges_n1["id"] = bridges_n1.index
bridges_n1["id"] = (bridges_n1["id"] + 1_000_000).astype(int)

# Add necessary default values for additional columns
bridges_n1['length'] = 0  # Default length for all bridges (adjust as needed)
bridges_n1['model_type'] = "bridge"  # Standardize as "bridge"

# Remove unnecessary columns from bridges to match roads (if any)
if 'lrp' in bridges_n1.columns:
    bridges_n1 = bridges_n1.drop(columns=['lrp'])
if 'gap' in bridges_n1.columns:
    bridges_n1 = bridges_n1.drop(columns=['gap'])

bridges_n1["id"] = bridges_n1.index
bridges_n1["id"] = (bridges_n1["id"] + 1_000_000).astype(int)
# Append bridges to the roads DataFrame

# Save the updated DataFrame back to the CSV file
# roads.to_csv("../data/cleaned_data/infrastructure/updated_roads.csv", index=False)

# Display the first few rows for verification
print(roads.head(40))


   road  chainage      lrp        lat        lon  gap            type  \
0    N1     0.000     LRPS  23.706028  90.443333  NaN          Others   
1    N1     0.814    LRPSa  23.702917  90.450417  NaN         Culvert   
2    N1     0.822    LRPSb  23.702778  90.450472  NaN       CrossRoad   
3    N1     1.000   LRP001  23.702139  90.451972  NaN          KmPost   
4    N1     2.000   LRP002  23.697889  90.460583  NaN          KmPost   
5    N1     2.130  LRP002a  23.697361  90.461667  NaN         Culvert   
6    N1     3.000   LRP003  23.693833  90.469138  NaN          KmPost   
7    N1     4.000   LRP004  23.693611  90.478777  NaN          KmPost   
8    N1     4.175  LRP004a  23.693805  90.480527  NaN  SideRoad,Right   
9    N1     5.000   LRP005  23.694750  90.488500  NaN          KmPost   
10   N1     6.000   LRP006  23.695916  90.498194  NaN          KmPost   
11   N1     7.000   LRP007  23.697333  90.507750  NaN          KmPost   
12   N1     7.181  LRP007a  23.697916  90.509278  N

In [5]:
bridges["type"] = bridges["type"].astype(str).str.lower()
b = bridges["type"].unique()
b = b.tolist()
b.append("culvert")
b.append("bridge")
b = np.array(b)
print(b)


['box culvert' 'pc girder bridge' 'pc box' 'rcc girder bridge'
 'slab culvert' 'steel beam & rcc slab' 'arch masonry' 'rcc bridge'
 'truss with rcc slab' 'baily with steel deck' 'truss with steel deck'
 'baily with timber deck' 'truss with timber deck' 'pipe culvert'
 'culvert' 'bridge']


In [6]:
# Rename and reformat the columns
roads["length"] = roads["chainage"].diff().abs().fillna(0)
roads["type"] = roads["type"].astype(str).str.lower()

# Update 'type' column values to standardize naming
roads.loc[roads["type"].isin(b), "type"] = "bridge"
roads.loc[roads["type"] != "bridge", "type"] = "link"

# Generate unique IDs for each road segment
roads["id"] = roads.index
roads["id"] = (roads["id"] + 1_000_000).astype(int)

# Correct assignment for model_type using .loc for single-element assignment
roads.loc[0, "type"] = "source"  # Assign 'source' to the first row
roads.loc[len(roads.index) - 1, "type"] = "sink"  # Assign 'sink' to the last row

# Rename column 'type' to 'model_type'
roads = roads.rename(columns={
    "type": "model_type",
})

# Drop unnecessary columns
roads = roads.drop('lrp', axis=1)
roads = roads.drop('gap', axis=1)

# Save the formatted data to a CSV file
roads.to_csv("../data/cleaned_data/infrastructure/roads_for_model.csv", index=False)

# Display the first 40 rows for verification
roads.head(40)

,road,chainage,lat,lon,model_type,name,length,id
0,N1,0.000,23.706028,90.443333,source,Start of Road after Jatrabari Flyover infront...,0.000,1000000
1,N1,0.814,23.702917,90.450417,bridge,Box Culvert,0.814,1000001
2,N1,0.822,23.702778,90.450472,link,Intersection with Z1101,0.008,1000002
3,N1,1.000,23.702139,90.451972,link,Km post missing,0.178,1000003
4,N1,2.000,23.697889,90.460583,link,Km post missing,1.000,1000004
5,N1,2.130,23.697361,90.461667,bridge,Box culvert,0.130,1000005
6,N1,3.000,23.693833,90.469138,link,Km post missing,0.870,1000006
7,N1,4.000,23.693611,90.478777,link,Km post missing,1.000,1000007
8,N1,4.175,23.693805,90.480527,link,Road to Narayanganj(R111),0.175,1000008
9,N1,5.000,23.694750,90.488500,link,Km post missing,0.825,1000009


In [7]:
# Step 3: Append it to the roads DataFrame
roads = pd.concat([roads, bridges_n1], ignore_index=True)

# Step 4: Sort the updated DataFrame by chainage
roads = roads.sort_values(by='chainage').reset_index(drop=True)

,road,chainage,lat,lon,model_type,name,length,id,km,LRPName,...,structureNr,roadName,width,constructionYear,spans,zone,circle,division,sub-division,EstimatedLoc
0,Z1444,0.0,23.006278,91.358278,link,Intersection with N104 at Temohoni.Feni,12.800,1025410,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R180,0.0,21.427249,92.006139,link,"Road start from N110 at bus terminal more, Cox...",17.270,1011677,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R861,0.0,23.187000,90.331888,link,Start of road from R860 at Monoharpur more.,74.634,1018897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Z1060,0.0,23.746444,90.472305,link,The road starts from R112,11.204,1022035,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R685,0.0,24.374444,88.604028,link,Start from Monument of Bindur more.,1.330,1016847,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R682,0.0,24.591555,88.282027,link,Start of road,13.230,1016841,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Z5301,0.0,26.197611,88.554111,link,Start at Boda Bus stand more near shahid minne...,27.960,1037060,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Z1013,0.0,22.000250,92.236194,link,Start of Road From Tonkabati,15.304,1020080,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Z8419,0.0,23.136111,89.920083,link,Start of road from R850 at Satpar,6.500,1049187,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,R748,0.0,23.645305,88.845055,link,Start from Boro bazar more infront of traffic ...,43.468,1017655,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
